<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/interactive_cancellations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive Visualization

In [ ]:
# Upgrade Matplotlib
!pip install matplotlib --upgrade
!pip install plotly --upgrade

In [ ]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact
#setting plot size
#plt.rcParams["figure.figsize"] = (7.0,4.0)

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [ ]:
# Storing loaded data from excel to a pandas dataframe
import io
df1 = pd.read_excel(io.BytesIO(uploaded1['2022_11_01_op.xlsx']))

In [ ]:
# Chainging columns to lower case
df1.rename(columns={i:i.lower() for i in df1.columns}, inplace=True)

In [ ]:
# Checking the dataframe info
df1.info()

In [ ]:
df1.head()

In [ ]:
# Splitting columns to lists for interactive visualization
date_col = df1.select_dtypes(include=['datetime64[ns]']).columns.tolist()
dim_col = df1.select_dtypes(include=['object']).columns.tolist()
eve_col = [i for i in dim_col if ('event') in i]
cat_col = [i for i in dim_col if ('category') in i]
num_col = df1.select_dtypes(include=['float64', 'int64']).columns.tolist()

In [ ]:
df1 = df1.assign(maturity_category = pd.cut(df1['lifetime_days'],3,['early','medium','late']))

In [ ]:
df1['maturity_category']

## Preparing data

In [ ]:
# Preparing first source table
df_temp1 = df1.groupby(by=['loyalty_stage_category','event_category']).agg(value = ('name','count')).reset_index()
df_temp1.rename(columns={'loyalty_stage_category':'source','event_category':'target'}, inplace=True)
df_temp1

In [ ]:
# Preparing second source table
df_temp2 = df1.groupby(by=['event_category','operator_out']).agg(value = ('name','count')).reset_index()
df_temp2.rename(columns={'event_category':'source','operator_out':'target'}, inplace=True)
df_temp2

In [ ]:
# Concatenating the 2 temporary dataframes
links = pd.concat([df_temp1, df_temp2], axis=0).reset_index(drop=True)
links

In [ ]:
# Generating a list of unique values present in source and target columns
unique_source_target = list(pd.unique(links[['source','target']].values.ravel('K')))
unique_source_target

In [ ]:
# Mapping a dictionary
mapping_dict =  {k:v for v,k in enumerate(unique_source_target)}
mapping_dict

In [ ]:
# Generating links mapped with indexes
links2 = links.copy()
links2 = links2.assign(source = links['source'].map(mapping_dict),
                       target = links['target'].map(mapping_dict))
links2

In [ ]:
# Generating list of dictionary
links_dict = links2.to_dict(orient='list')
links_dict

## Sankey Visualization

In [ ]:
# Generating Sankey Diagram
fig_1 = go.Figure(
    data=[go.Sankey(
        # Setting the data for the node
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color='black',width=0.5),
            label=unique_source_target,
            color=['#a6a6a6','#ffff00','#00b050','#ff0000','#ffc000','#0080FF','#b4b404','#64b23b',
                   '#a2a2a2','#cc071e','#79d64a','#01acfb','#990ae3','#15150c']
#3c3766]
        ),
        # Setting the data for the link
        link = dict(
            source=links_dict['source'],
            target=links_dict['target'],
            value=links_dict['value']
        )
    )]
)

In [ ]:
# Plotting the Sankey Diagram
fig_1.update_layout(title_text='Loyalty, Churn and Operators Diagram - October 2022', font_size=10)
fig_1.show()